In [1]:
import pandas as pd
import os
import wrangle
import matplotlib as plt
import seaborn as sns
import numpy as np
from env import host, user, password
from wrangle import wrangle_zillow

In [ ]:
def get_connection(db, username=user, host=host, password=password):
    return f'mysql+pymysql://{username}:{password}@{host}/{db}'

## Aquiring the data

In [ ]:
sql = """
        SELECT prop.*,
        predictions_2017.logerror,
        predictions_2017.transactiondate,
        air.airconditioningdesc,
        arch.architecturalstyledesc,
        build.buildingclassdesc,
        heat.heatingorsystemdesc,
        land.propertylandusedesc,
        story.storydesc,
        type.typeconstructiondesc
        FROM properties_2017 prop
        JOIN (
            SELECT parcelid, MAX(transactiondate) AS max_transactiondate
            FROM predictions_2017
            GROUP BY parcelid
            ) pred USING(parcelid)
        JOIN predictions_2017 ON pred.parcelid = predictions_2017.parcelid
                          AND pred.max_transactiondate = predictions_2017.transactiondate
        LEFT JOIN airconditioningtype air USING(airconditioningtypeid)
        LEFT JOIN architecturalstyletype arch USING(architecturalstyletypeid)
        LEFT JOIN buildingclasstype build USING(buildingclasstypeid)
        LEFT JOIN heatingorsystemtype heat USING(heatingorsystemtypeid)
        LEFT JOIN propertylandusetype land USING(propertylandusetypeid)
        LEFT JOIN storytype story USING(storytypeid)
        LEFT JOIN typeconstructiontype type USING(typeconstructiontypeid)
        WHERE propertylandusedesc = "Single Family Residential"
            AND transactiondate <= '2017-12-31'
            AND prop.longitude IS NOT NULL
            AND prop.latitude IS NOT NULL
    """

df = pd.read_sql(sql, get_connection("zillow"));

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
pd.set_option('display.max_columns', None)

df.head()

In [ ]:
df.shape

## Nulls of rows

In [ ]:

nulls_col = pd.DataFrame({'num_rows_missing': df.isnull().sum(), 
              'percent_rows_missing': (df.isnull().sum() / df.shape[0] * 100)})
pd.set_option('display.max_columns', None)
nulls_col.sort_values(by='num_rows_missing', ascending=True).head(10)

In [ ]:
def nulls_by_col(df):
    num_missing = df.isnull().sum()
    rows = df.shape[0]
    prcnt_miss = num_missing / rows * 100
    cols_missing = pd.DataFrame({'num_rows_missing': num_missing, 'percent_rows_missing': prcnt_miss})
    return cols_missing.sort_values(by='num_rows_missing', ascending=True)


In [ ]:
Number of 

In [ ]:
nulls_by_col(df)

## Nulls of collums 

In [ ]:
nulls_row = pd.DataFrame({'num_cols_missing': df.isnull().sum(axis=1),
              'percent_cols_missing': df.isnull().sum(axis=1)/df.shape[1]*100})

nulls_row

- honestly there is so much missing in my cluster excercizez imma keep it simple and just do what i did and keep 60% of the data 

In [ ]:
def nulls_by_row(df):
    num_missing = df.isnull().sum(axis=1)
    prcnt_miss = num_missing / df.shape[1] * 100
    rows_missing = pd.DataFrame({'num_cols_missing': num_missing, 'percent_cols_missing': prcnt_miss})
    rows_missing = df.merge(rows_missing,
                        left_index=True,
                        right_index=True)[['parcelid', 'num_cols_missing', 'percent_cols_missing']]
    return rows_missing.sort_values(by='num_cols_missing', ascending=False)

In [ ]:
def summarize(df):
    '''
    summarize will take in a single argument (a pandas dataframe) 
    and output to console various statistics on said dataframe, including:
    # .head()
    # .info()
    # .describe()
    # .value_counts()
    # observation of nulls in the dataframe
    '''
    print('SUMMARY REPORT')
    print('=====================================================\n\n')
    print('Dataframe head: ')
    print(df.head(3))
    print('=====================================================\n\n')
    print('Dataframe info: ')
    print(df.info())
    print('=====================================================\n\n')
    print('Dataframe Description: ')
    print(df.describe())
    num_cols = [col for col in df.columns if df[col].dtype != 'O']
    cat_cols = [col for col in df.columns if col not in num_cols]
    print('=====================================================')
    print('DataFrame value counts: ')
    for col in df.columns:
        if col in cat_cols:
            print(df[col].value_counts(), '\n')
        else:
            print(df[col].value_counts(bins=10, sort=False), '\n')
    print('=====================================================')
    print('nulls in dataframe by column: ')
    print(nulls_by_col(df))
    print('=====================================================')
    print('nulls in dataframe by row: ')
    print(nulls_by_row(df))
    print('=====================================================')


In [ ]:
summarize(df)


## now im going to drop collumns and rows

In [ ]:
def remove_columns(df, cols_to_remove):
    df = df.drop(columns=cols_to_remove)
    return df

In [ ]:
def handle_missing_values(df, prop_required_columns=0.6, prop_required_row=0.75):
    threshold = int(round(prop_required_columns * len(df.index), 0))
    df = df.dropna(axis=1, thresh=threshold) #1, or ‘columns’ : Drop columns which contain missing value
    threshold = int(round(prop_required_row * len(df.columns), 0))
    df = df.dropna(axis=0, thresh=threshold) #0, or ‘index’ : Drop rows which contain missing values.
    return df

In [ ]:
def drop_r_nulls(df):
    df = df.dropna()
    return df

In [ ]:
def data_prep(df, cols_to_remove=[], prop_required_column=0.6, prop_required_row=0.75):
    df = remove_columns(df, cols_to_remove)
    df = handle_missing_values(df, prop_required_column, prop_required_row)
    df = drop_r_nulls(df)
    return df

In [ ]:
df = data_prep(df)

In [ ]:
df.info()

In [ ]:
data_prep(df).to_csv('zillow.csv')

In [ ]:
df2 = pd.read_csv('zillow.csv')
df2.head()

In [ ]:
df2.shape

In [ ]:
df.info()

In [ ]:
get_zillow_data(df)

In [2]:
df = wrangle_zillow()

In [3]:
df.head()

,Unnamed: 0,id,parcelid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,...,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock,logerror,transactiondate,heatingorsystemdesc,propertylandusedesc
0,3,2288172,12177905,3.0,4.0,8.0,3.0,2376.0,2376.0,6037.0,...,108918.0,145143.0,2016.0,36225.0,1777.51,6.037300e+13,-0.103410,2017-01-01,Central,Single Family Residential
1,4,781532,12095076,3.0,4.0,9.0,3.0,2962.0,2962.0,6037.0,...,276684.0,773303.0,2016.0,496619.0,9516.26,6.037461e+13,-0.001011,2017-01-01,Central,Single Family Residential
2,6,1246926,12790562,3.0,4.0,9.0,3.0,3039.0,3039.0,6037.0,...,177527.0,220583.0,2016.0,43056.0,3104.19,6.037500e+13,-0.040966,2017-01-02,Central,Single Family Residential
3,7,1639362,11104527,3.0,4.0,8.0,3.0,2540.0,2540.0,6037.0,...,271715.0,430108.0,2016.0,158393.0,6103.36,6.037920e+13,0.005963,2017-01-02,Central,Single Family Residential
4,11,571930,11830315,1.0,2.0,4.0,1.0,1146.0,1146.0,6037.0,...,127960.0,273745.0,2016.0,145785.0,3739.02,6.037532e+13,-0.035085,2017-01-02,Floor/Wall,Single Family Residential


In [4]:
def drop_one(df):
    df = df.drop(columns=['Unnamed: 0'])
    return df
    

In [5]:
drop_one(df)


,id,parcelid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,fullbathcnt,...,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock,logerror,transactiondate,heatingorsystemdesc,propertylandusedesc
0,2288172,12177905,3.0,4.0,8.0,3.0,2376.0,2376.0,6037.0,3.0,...,108918.0,145143.0,2016.0,36225.0,1777.51,6.037300e+13,-0.103410,2017-01-01,Central,Single Family Residential
1,781532,12095076,3.0,4.0,9.0,3.0,2962.0,2962.0,6037.0,3.0,...,276684.0,773303.0,2016.0,496619.0,9516.26,6.037461e+13,-0.001011,2017-01-01,Central,Single Family Residential
2,1246926,12790562,3.0,4.0,9.0,3.0,3039.0,3039.0,6037.0,3.0,...,177527.0,220583.0,2016.0,43056.0,3104.19,6.037500e+13,-0.040966,2017-01-02,Central,Single Family Residential
3,1639362,11104527,3.0,4.0,8.0,3.0,2540.0,2540.0,6037.0,3.0,...,271715.0,430108.0,2016.0,158393.0,6103.36,6.037920e+13,0.005963,2017-01-02,Central,Single Family Residential
4,571930,11830315,1.0,2.0,4.0,1.0,1146.0,1146.0,6037.0,1.0,...,127960.0,273745.0,2016.0,145785.0,3739.02,6.037532e+13,-0.035085,2017-01-02,Floor/Wall,Single Family Residential
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32040,1373391,10722691,2.0,3.0,6.0,2.0,1570.0,1570.0,6037.0,2.0,...,46784.0,72026.0,2016.0,25242.0,1000.70,6.037135e+13,0.081196,2017-09-19,Central,Single Family Residential
32041,2274245,12412492,2.0,4.0,6.0,2.0,1633.0,1633.0,6037.0,2.0,...,125466.0,346534.0,2016.0,221068.0,4175.08,6.037555e+13,0.001082,2017-09-19,Central,Single Family Residential
32042,673515,11000655,2.0,2.0,6.0,2.0,1286.0,1286.0,6037.0,2.0,...,70917.0,354621.0,2016.0,283704.0,4478.43,6.037101e+13,0.020615,2017-09-20,Central,Single Family Residential
32043,1843709,12773139,1.0,3.0,4.0,1.0,1032.0,1032.0,6037.0,1.0,...,32797.0,49546.0,2016.0,16749.0,876.43,6.037434e+13,0.037129,2017-09-21,Central,Single Family Residential


In [ ]:
df = df.drop(columns=['Unnamed: 0'])



In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.size

In [ ]:
df.shape

In [ ]:
df = get_zillow_data()